<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/finQA_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install huggingface
!pip install transformers
!pip install langchain

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
## Navigating to the Project Directory
%cd drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [5]:
%ls

CodeBase/  Dataset/  Papers/  README.md


In [6]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 6.0 MB/s eta 0:00:00


In [34]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Dataset/etr_sampled_earning_report.pdf")
pages = loader.load()

In [35]:
len(pages)

2

In [43]:
pages

[Document(page_content='ENTERGY CORPORATION AND SUBSIDIARIES\nMANAGEMENT’S FINANCIAL DISCUSSION AND ANALYSIS\nEntergy operates primarily through two business segments: Utility and Entergy Wholesale Commodities.\n• The Utility  business segment includes the generation, transmission, distribution, and sale of electric power in\nportions of Arkansas, Mississippi, Texas, and Louisiana, including the City of New Orleans; and operation of\na small natural gas distribution business.\n• The Entergy Wholesale Commodities  business segment includes the ownership, operation, and\ndecommissioning of nuclear power plants located in the northern United States and the sale of the electric\npower produced by its operating plants to wholesale customers.  Entergy Wholesale Commodities also provides\nservices to other nuclear power plant owners and owns interests in non-nuclear power plants that sell the\nelectric power produced by those plants to wholesale customers.  See “ Entergy Wholesale Commodities

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

In [64]:
docs=r_splitter.split_documents(pages)

In [65]:
for chunk in docs:
  print(len(chunk.page_content))

408
350
380
420
332
361
95
434
422
440
347
355
352
349
366
228


In [45]:
!pip install unstructured libmagic python-magic python-magic-bin

  Using cached libmagic-1.0.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [46]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c8209a74e753d615e9f1fb770e9ef937af60117f2f7831f45d87837844b0b000
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [49]:
from sentence_transformers import SentenceTransformer

In [50]:
encoder = SentenceTransformer("all-mpnet-base-v2")

In [66]:
docs

[Document(page_content='ENTERGY CORPORATION AND SUBSIDIARIES\nMANAGEMENT’S FINANCIAL DISCUSSION AND ANALYSIS\nEntergy operates primarily through two business segments: Utility and Entergy Wholesale Commodities.\n• The Utility  business segment includes the generation, transmission, distribution, and sale of electric power in\nportions of Arkansas, Mississippi, Texas, and Louisiana, including the City of New Orleans; and operation of', metadata={'source': 'Dataset/etr_sampled_earning_report.pdf', 'page': 0}),
 Document(page_content='a small natural gas distribution business.\n• The Entergy Wholesale Commodities  business segment includes the ownership, operation, and\ndecommissioning of nuclear power plants located in the northern United States and the sale of the electric\npower produced by its operating plants to wholesale customers.  Entergy Wholesale Commodities also provides', metadata={'source': 'Dataset/etr_sampled_earning_report.pdf', 'page': 0}),
 Document(page_content='service

In [53]:
import numpy as np

In [67]:
vectors=np.array([encoder.encode(chunk.page_content) for chunk in docs])

In [68]:
dim=vectors.shape[1]


In [69]:
import faiss

index = faiss.IndexFlatL2(dim)

In [70]:
index.add(vectors)

In [71]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7b2bd659a130> >

In [74]:
len(docs)==vectors.shape[0]

True

In [75]:
search_query="What is the percetage of total assets in utility sector in 2016?"


In [76]:
vec = encoder.encode(search_query)
vec.shape

(768,)

In [77]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)